In [1]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
from pathlib import Path

In [3]:
import ecephys_analyses as ea
from ecephys.sglx_utils import load_trigger
import ecephys.signal.timefrequency as tfr

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
def get_spectrogram(sig, **kwargs):
    nperseg = int(4 * sig.fs) # 4 second window
    noverlap = nperseg // 4 # 1 second overlap
    spg = tfr.parallel_spectrogram_welch(sig, nperseg=nperseg, noverlap=noverlap, **kwargs)
    return spg.median(dim="channel").sel(frequency=slice(0, 300))

In [6]:
def get_condition_spgs(subject, experiment, condition, probe):
    cx_chans = ea.get_channels(subject, experiment, probe, "superficial_ctx", asarray=False)
    wm_chans = ea.get_channels(subject, experiment, probe, "white_matter", asarray=False)
    
    bin_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "lf.bin")
    sr_chans_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "sr_chans.csv")
    spg_paths = ea.get_sglx_style_datapaths(subject, experiment, condition, "spg.nc")
    
    for bin_path, sr_chans_path, spg_path in zip(bin_paths, sr_chans_paths, spg_paths):
        sr_chans_df = ea.load_sr_chans(sr_chans_path)
        epochs = list()
        for epoch in sr_chans_df.itertuples():
            if not epoch.sr_chans:
                continue
            cx = load_trigger(bin_path, cx_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            wm = load_trigger(bin_path, wm_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            sr = load_trigger(bin_path, epoch.sr_chans, start_time=epoch.start_time, end_time=epoch.end_time)

            cx_wm_ref = cx - wm.values
            sr_wm_ref = sr - wm.values

            spgs = xr.Dataset({'mpta_wm_ref': get_spectrogram(cx_wm_ref), 
                               'sr_wm_ref': get_spectrogram(sr_wm_ref)})
            epochs.append(spgs)
            
        if not epochs:
            continue
        spgs = xr.concat(epochs, dim="time")
        Path(spg_path).parent.mkdir(parents=True, exist_ok=True) # Create parent directories if they do not already exist.
        spgs.to_netcdf(spg_path)
        spgs.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [7]:
get_condition_spgs("Segundo", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:21:27: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t8.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:25:39: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t9.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:26:27: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t10.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
16:29:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX2-Segundo/1-21-2020/1-21-2020_g0/1-21-2020_g0_imec0/1-21-2020_g0_t11.imec0.lf.bin
nChan: 385, nFileSamp: 1800000

In [8]:
get_condition_spgs("Valentino", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:50:34: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:54:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX3-Valentino/2-19-2020/2-19-2020_g1/2-19-2020_g1_imec0/2-19-2020_g1_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
nChan: 385, nFileSamp: 18000000
17:58:50: Finished /Volumes/neur

In [9]:
get_condition_spgs("Doppio", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
19:40:32: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
19:44:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX4-Doppio/3-17-2020/3-17-2020_g0/3-17-2020_g0_imec0/3-17-2020_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
nChan: 385, nFileSamp: 18000019
19:48:47: Finished /Volumes/neuropixel

In [13]:
get_condition_spgs("Alessandro", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 9000051
nChan: 385, nFileSamp: 9000051
nChan: 385, nFileSamp: 9000051
19:17:27: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/8-24-2020/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
19:19:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/8-24-2020/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
19:21:05: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX5-Alessandro/8-24-2020/8-24-2020_g0/8-24-2020_g0_imec0/8-24-2020_g0_t2.imec0.lf.bin
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 9000052
nChan: 385, nFileSamp: 900

In [11]:
get_condition_spgs("Allan", "sleep-homeostasis", "all", "imec1")

nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
nChan: 385, nFileSamp: 4242731
11:20:29: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_g0/3-1-2021_g0_imec1/3-1-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 361262
nChan: 385, nFileSamp: 361262
nChan: 385, nFileSamp: 361262
11:20:38: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021-a_g0/3-1-2021-a_g0_imec1/3-1-2021-a_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 3863847
nChan: 385, nFileSamp: 3863847
nChan: 385, nFileSamp: 3863847
11:21:39: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX8-Allan/3-1-2021/3-1-2021_A_g0/3-1-2021_A_g0_imec1/3-1-2021_A_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
nChan: 385, nFileSamp: 8999998
11:23:50: Fin

In [12]:
get_condition_spgs("Eugene", "sleep-homeostasis", "all", "imec0")

nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
13:46:12: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs_g0/9.24.2020_BL_24hs_g0_imec0/9.24.2020_BL_24hs_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
nChan: 385, nFileSamp: 9000025
13:51:06: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t0.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
13:55:56: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020_BL_24hs2_g0_imec0/9.24.2020_BL_24hs2_g0_t1.imec0.lf.bin
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
nChan: 385, nFileSamp: 9000026
14:00:46: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX6-Eugene/9-24-2020/9.24.2020_BL_24hs2_g0/9.24.2020